Post Process tool for simulated logs


In [1]:
#import necesary libraries
import pandas as pd
import io
import numpy as np
import seaborn as sns
import earthpy as et

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Use white grid plot background from seaborn
sns.set(font_scale=1.5, style="whitegrid")
import matplotlib.pyplot as plt


#simulated
data1 = pd.read_csv(r'D:\OneDrive\OneDrive - WFT\Compartido\Well_Datasets\La_Calera_Pluspetrol\Post_Process\sim_logs\lc-3008\Log20032021.csv')


print("Data Loaded OK")



FileNotFoundError: [Errno 2] File b'D:\\OneDrive\\OneDrive - WFT\\Compartido\\Well_Datasets\\La_Calera_Pluspetrol\\Post_Process\\sim_logs\\lc-3008\\Log20032021.csv' does not exist: b'D:\\OneDrive\\OneDrive - WFT\\Compartido\\Well_Datasets\\La_Calera_Pluspetrol\\Post_Process\\sim_logs\\lc-3008\\Log20032021.csv'

cleaning and packing data

In [98]:
data1[['Time', 'Pressure[Bar]', 'DP[Bar]', 'Temperature[C]',
       'Velocity[m/s]', 'Quality',
       'WaterCut[%]', 'Qg[m3/s]', 'Ql[m3/s]', 'Qo[m3/s]',
       'Qw[m3/s]', 'QgStd[m3/s]', 'QlStd[m3/s]', 'QoStd[m3/s]', 'QwStd[m3/s]',
       'TotWLR[%]', 'TotGOR', 'GVF[%]','GasDen[kg/m3]','GasVisc[cP]', 
       'LiqDen[kg/m3]', 'LiqVisc[cP]', 
       'PVT_Var', 'MVT_Var_out', 'Sonar_Vel_Var', 
       'PI_Status_Out', 'ErrQt', 'Vm[m/s]', 
       'diffPress_Input[bar]', 'diffPressZero[bar]', 'ParamFile', 'PVTFile',
       'Rhog', 'Rhol', 'Rhoo', 'Rhow'
       ]].info()


#filetering columns

ds_Q = data1[['Time', 'Pressure[Bar]', 'DP[Bar]', 'Temperature[C]',
       'Velocity[m/s]', 'Quality',
       'WaterCut[%]', 'Qg[m3/s]', 'Ql[m3/s]', 'Qo[m3/s]',
       'Qw[m3/s]', 'QgStd[m3/s]', 'QlStd[m3/s]', 'QoStd[m3/s]', 'QwStd[m3/s]',
       'TotWLR[%]', 'TotGOR', 'GVF[%]','GasDen[kg/m3]','GasVisc[cP]', 
       'LiqDen[kg/m3]', 'LiqVisc[cP]', 
       'PVT_Var', 'MVT_Var_out', 'Sonar_Vel_Var', 
       'PI_Status_Out', 'ErrQt', 'Vm[m/s]', 
       'diffPress_Input[bar]', 'diffPressZero[bar]', 'ParamFile', 'PVTFile',
       'Rhog', 'Rhol', 'Rhoo', 'Rhow'
       ]]

#adding columns with 24hs rates

ds_Q.loc[:,"Qg[m3/d]"] = ds_Q.loc[:,"Qg[m3/s]"] * 86400
ds_Q.loc[:,"Ql[m3/d]"] = ds_Q.loc[:,"Ql[m3/s]"] * 86400
ds_Q.loc[:,"Qo[m3/d]"] = ds_Q.loc[:,"Qo[m3/s]"] * 86400
ds_Q.loc[:,"Qw[m3/d]"] = ds_Q.loc[:,"Qw[m3/s]"] * 86400
ds_Q.loc[:,"Gas Rate Sm3/d"] = ds_Q.loc[:,"QgStd[m3/s]"] * 86400
ds_Q.loc[:,"QlStd[m3/d]"] = ds_Q.loc[:,"QlStd[m3/s]"] * 86400
ds_Q.loc[:,"Oil Flow Sm3/d"] = ds_Q.loc[:,"QoStd[m3/s]"] * 86400
ds_Q.loc[:,"Water Flow Rate m3/d"] = ds_Q.loc[:,"QwStd[m3/s]"] * 86400

#cleaning columns

to_drop = ['Qg[m3/s]', 'Ql[m3/s]', 'Qo[m3/s]',
       'Qw[m3/s]', 'QgStd[m3/s]', 'QlStd[m3/s]', 'QoStd[m3/s]', 'QwStd[m3/s]']

ds_Qf= ds_Q.drop(to_drop, axis=1)


ds = ds_Qf.copy()
#ds1 = ds_data.groupby('DATE & TIME', as_index=True)
ds = ds.set_index('Time')
#ds2 = ds_Qf.set_index('Time').groupby(pd.Grouper(freq='17min')).mean()

ds.index = pd.to_datetime(ds.index)
#ds.index

ds2 = ds.groupby(pd.Grouper(freq='1min')).mean() 
ds2.reset_index(level=0, inplace=True)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 36 columns):
Time                    910 non-null object
Pressure[Bar]           910 non-null float64
DP[Bar]                 910 non-null float64
Temperature[C]          910 non-null float64
Velocity[m/s]           910 non-null float64
Quality                 910 non-null float64
WaterCut[%]             910 non-null int64
Qg[m3/s]                910 non-null float64
Ql[m3/s]                910 non-null float64
Qo[m3/s]                910 non-null float64
Qw[m3/s]                910 non-null float64
QgStd[m3/s]             910 non-null float64
QlStd[m3/s]             910 non-null float64
QoStd[m3/s]             910 non-null float64
QwStd[m3/s]             910 non-null float64
TotWLR[%]               910 non-null float64
TotGOR                  910 non-null float64
GVF[%]                  910 non-null float64
GasDen[kg/m3]           910 non-null float64
GasVisc[cP]             910 non-null float6

Adding meter and well information

G2 = ['LCA-3083','LCA-3084','LCA-3085']
G7 = ['LCA-3005', 'LCA-3006', 'LCA-3007']
G8 = ['LCA-3008', 'LCA-3009', 'LCA-3010']
K3 = ['LCA-3011', 'LCA-3012', 'LCA-3013']
lc3001 = ['LCA-3001']


In [99]:
ds2['DATE & TIME'] = ds2['Time']
ds2['Test'] = 'sim1'
ds2['Gas Spec. Gravity air=1'] = ''
ds2['Well Head Pressure  psig'] =''
ds2['Well Head Temp deg C']=''
ds2['Sep. Static Pressure psig']=''
ds2['Sep. Gas Temp deg C'] = ''
ds2['WELL Name'] = 'LCA-3008'
ds2['CHOKE'] = '40'
ds2['Gas Cumm. Sm3'] = ''
ds2['GOR (Gas/Oil Ratio) m3/m3'] = ds2['TotGOR']
ds2['Oil Density  g/cm3'] = ''
ds2['Oil Temp deg C'] = ''
ds2[' Raw Oil Flow  m3/d'] = ds2['Qo[m3/d]']
ds2['WaterDensity Kg/m3'] = ''
ds2['Flow Water Cut  %'] = ds2['TotWLR[%]']

column order to merge

In [100]:
d = ds2[['DATE & TIME','CHOKE','WELL Name', 'Test', 'Well Head Pressure  psig', 'Well Head Temp deg C',
         'Sep. Static Pressure psig', 'Sep. Gas Temp deg C', 'Gas Spec. Gravity air=1','Gas Rate Sm3/d',
        'Gas Cumm. Sm3','GOR (Gas/Oil Ratio) m3/m3','Oil Density  g/cm3','Oil Temp deg C',
         ' Raw Oil Flow  m3/d','Oil Flow Sm3/d','Water Flow Rate m3/d','WaterDensity Kg/m3',
        'Flow Water Cut  %'
        ]]

export to csv

In [101]:
d.to_csv(r'D:\OneDrive\OneDrive - WFT\Compartido\Well_Datasets\La_Calera_Pluspetrol\Post_Process\append\sim1_lc_3008.csv')

append files


In [37]:
#TO combine multiple files

import glob
import pandas as pd

# get data file names
path = r'D:\OneDrive\OneDrive - WFT\Compartido\Well_Datasets\La_Calera_Pluspetrol\Post_Process\append'
filenames = glob.glob(path + "\*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename,low_memory=False).assign(filename = filename))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True,sort=False)

big_frame.to_csv(r"D:\OneDrive\OneDrive - WFT\Compartido\Well_Datasets\La_Calera_Pluspetrol\Post_Process\append\raw_full.csv", index=False, encoding='utf-8-sig')